In [ ]:
column_names = ['timestamp'] + [name.lower() + '_' + char for name in landmark_names for char in ('x', 'y', 'z')]

data = pd.DataFrame(columns = column_names, dtype = float)

time = 0
cap = cv2.VideoCapture(source)

# Setup MediaPipe instance
with mp_pose.Pose(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    enable_segmentation=True
) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        image_shape = image.shape


        # Camera condition -> if selfie or web camera is chosen then flip image
        if source == 0:
            image = cv2.flip(image, 1)
    
        # Recolor image for image processing
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Image MediaPipe processing -> detection
        results = pose.process(image)

        # Recolor back to BGR for visualization
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a fake background
        fake_background = np.zeros(shape=image_shape, dtype='uint8')
        # use a fake background if necessary
        if not background:
            image = fake_background

        # Image processing for different outputs
        if output == 'landmarks':
            default_landmarks = results.pose_landmarks

            if default_landmarks:
                time += 1

                # Get custom landmarks and create new connection
                custom_landmarks = get_custom_landmarks(chosen_indexes, default_landmarks)
                #custom_connections.add((0, len(custom_landmarks.landmark) - 1))

                # Prepare a single record storage
                record = np.array([time])

                for landmark in custom_landmarks.landmark:
                    # Extract pose landmarks coordinates and store as array
                    coordinates = landmark2array(landmark)[:3]
                    record = np.concatenate(
                        [
                            record,
                            coordinates
                        ]
                    )

                # Save pose landmarks coordinates in time as DataFrame
                data = pd.concat([data, pd.DataFrame([record], columns = column_names)], ignore_index=True)

                # Draw customize landmarks on image
                mp_drawing.draw_landmarks(
                    image,
                    landmark_list=custom_landmarks,
                    connections=connections,
                )


        elif output == 'mask':
            image = results.segmentation_mask
            if image is None:
                image = fake_background


        cv2.imshow(f'background: {background}, output: {output}', image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

        if not cap.isOpened():
            exit()
            
cap.release()
cv2.destroyAllWindows()